In [ ]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [ ]:
config = configparser.ConfigParser()
config.read('dl.cfg')

KEY    = config.get("AWS","AWS_ACCESS_KEY_ID")
SECRET = config.get("AWS","AWS_SECRET_ACCESS_KEY")
os.environ['AWS_ACCESS_KEY_ID']=config.get("AWS","AWS_ACCESS_KEY_ID")
os.environ['AWS_SECRET_ACCESS_KEY']=config.get("AWS","AWS_SECRET_ACCESS_KEY")

In [ ]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [ ]:
input_data = "s3a://udacity-dend/"

In [ ]:
# get filepath to song data file
song_data = input_data+'song_data'

# read song data file
df = spark.read.json(song_data)